In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

In [3]:
# creating a spark session
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/01 17:34:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# creating a mapper to parse the data row
def mapper(line):
    fields = line.split(",")
    return Row(ID=int(fields[0]), name=str(fields[1].encode("utf-8")), \
        age=int(fields[2]), numFriends=int(fields[3]))

In [5]:
lines = spark.sparkContext.textFile("resources/fakefriends.csv")

In [6]:
people = lines.map(mapper)

In [7]:
# inferring the schema, and register the dataframe as a table
schemaPeople = spark.createDataFrame(people).cache()
schemaPeople.createOrReplaceTempView("people")

In [11]:
schemaPeople.show(5)

+---+-----------+---+----------+
| ID|       name|age|numFriends|
+---+-----------+---+----------+
|  0|    b'Will'| 33|       385|
|  1|b'Jean-Luc'| 26|         2|
|  2|    b'Hugh'| 55|       221|
|  3|  b'Deanna'| 40|       465|
|  4|   b'Quark'| 68|        21|
+---+-----------+---+----------+
only showing top 5 rows



In [12]:
# SQL can be run over DataFrames that have been registered as a table
teenagers = spark.sql("SELECT * FROM people WHERE age >= 13 AND age <= 19")

In [13]:
for teen in teenagers.collect():
    print(teen)

Row(ID=21, name="b'Miles'", age=19, numFriends=268)
Row(ID=52, name="b'Beverly'", age=19, numFriends=269)
Row(ID=54, name="b'Brunt'", age=19, numFriends=5)
Row(ID=106, name="b'Beverly'", age=18, numFriends=499)
Row(ID=115, name="b'Dukat'", age=18, numFriends=397)
Row(ID=133, name="b'Quark'", age=19, numFriends=265)
Row(ID=136, name="b'Will'", age=19, numFriends=335)
Row(ID=225, name="b'Elim'", age=19, numFriends=106)
Row(ID=304, name="b'Will'", age=19, numFriends=404)
Row(ID=341, name="b'Data'", age=18, numFriends=326)
Row(ID=366, name="b'Keiko'", age=19, numFriends=119)
Row(ID=373, name="b'Quark'", age=19, numFriends=272)
Row(ID=377, name="b'Beverly'", age=18, numFriends=418)
Row(ID=404, name="b'Kasidy'", age=18, numFriends=24)
Row(ID=409, name="b'Nog'", age=19, numFriends=267)
Row(ID=439, name="b'Data'", age=18, numFriends=417)
Row(ID=444, name="b'Keiko'", age=18, numFriends=472)
Row(ID=492, name="b'Dukat'", age=19, numFriends=36)
Row(ID=494, name="b'Kasidy'", age=18, numFriends=194)

In [15]:
# we can also use functions instread of SQL queries
schemaPeople.groupBy("age").count().orderBy("age").show()

+---+-----+
|age|count|
+---+-----+
| 18|    8|
| 19|   11|
| 20|    5|
| 21|    8|
| 22|    7|
| 23|   10|
| 24|    5|
| 25|   11|
| 26|   17|
| 27|    8|
| 28|   10|
| 29|   12|
| 30|   11|
| 31|    8|
| 32|   11|
| 33|   12|
| 34|    6|
| 35|    8|
| 36|   10|
| 37|    9|
+---+-----+
only showing top 20 rows



In [16]:
spark.stop()